# What is social distancing?
 ![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/05/social_distance_detector_example.png)
 -  Using computer vision technology based on OpenCV and YOLO-based deep learning, we are able to estimate the social distance of people in video streams. 

OpenCV ( Open Source Computer Vision )은 실시간 컴퓨터 비전을 목적으로 한 프로그래밍 라이브러리이다. 원래는 인텔이 개발하였다. 실시간 이미지 프로세싱에 중점을 둔 라이브러리이다.

## Using OpenCV, computer vision, and deep learning for social distancing
![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/05/social_distance_detector_steps.png)
...
- We can use OpenCV, computer vision, and deep learning to implement social distancing detectors.

- The steps to build a social distancing detector include:

1. Apply object detection to detect all people (and only people) in a video stream (see this tutorial on building an OpenCV people counter)

2. Compute the pairwise distances between all detected people

3. Based on these distances, check to see if any two people are less than N pixels apart

Our OpenCV social distancing detector implementation will rely on pixel distances 
...


## Project structure

```
OpenCV Social Distancing Detector
$ tree --dirsfirst
.
├── pyimagesearch
│   ├── __init__.py
│   ├── detection.py
│   └── social_distancing_config.py
├── yolo-coco
│   ├── coco.names
│   ├── yolov3.cfg
│   └── yolov3.weights
├── output.avi
├── pedestrians.mp4
└── social_distance_detector.py
2 directories, 9 files

```

Our YOLO object detector files including the CNN architecture definition, pre-trained weights, and class names are housed in the yolo-coco/ directory. This YOLO model is compatible with OpenCV’s DNN module.


## * social_distancing_config.py: A Python file holding a number of constants in one convenient place.
(from pyimagesearch import social_distancing_config as config)

In [51]:
# base path to YOLO directory
MODEL_PATH = "yolo-coco"

# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# boolean indicating if NVIDIA CUDA GPU should be used 
#(requires that OpenCV’s “dnn” module be installed with NVIDIA GPU support).
USE_GPU = False

# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50

### * detection.py :  YOLO object detection with OpenCV 
( from pyimagesearch.detection import detect_people)
- Our detect_people utility function, which detects people in video streams using the YOLO object detector

```
*detect_people*; the function accepts four parameters:

- frame: The frame from your video file or directly from your webcam
- net: The pre-initialized and pre-trained YOLO object detection model
- ln: The YOLO CNN output layer names
- personIdx: The YOLO model can detect many types of objects; this index is specifically for the person class, as we won’t be considering other objects

We then initialize our results list, which the function ultimately returns.
The results consist of (1) the person prediction probability, (2) bounding box coordinates for the detection, and (3) the centroid of the object.
```

In [52]:
# import the necessary packages
# from .social_distancing_config import NMS_THRESH
# from .social_distancing_config import MIN_CONF
import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter detections by (1) ensuring that the object
			# detected was a person and (2) that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	# return the list of results
	return results

### Social Distance Detector
(social_distance_detector.py)

In [53]:
#@title 기본 제목 텍스트
# USAGE
# python social_distance_detector.py --input pedestrians.mp4
# python social_distance_detector.py --input pedestrians.mp4 --output output.avi

# import the necessary packages
# from pyimagesearch import social_distancing_config as config
# from pyimagesearch.detection import detect_people
from scipy.spatial import distance as dist
import numpy as np
# import argparse
import imutils
import cv2
import os

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--input", type=str, default="",
# 	help="path to (optional) input video file")
# ap.add_argument("-o", "--output", type=str, default="",
# 	help="path to (optional) output video file")
# ap.add_argument("-d", "--display", type=int, default=1,
# 	help="whether or not output frame should be displayed")
# args = vars(ap.parse_args())

args = {'input':'pedestrians.mp4','output':'output.avi','display':0}


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
!pwd

/content


In [56]:
!cp '/content/drive/My Drive/yolo-coco.zip' ./  #(./) 현재위치로

In [57]:
!unzip yolo-coco.zip

Archive:  yolo-coco.zip
replace yolo-coco/coco.names? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: yolo-coco/coco.names    
replace yolo-coco/yolov3.cfg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: yolo-coco/yolov3.cfg    
replace yolo-coco/yolov3.weights? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: yolo-coco/yolov3.weights  y



In [58]:
%cd /content/

/content


In [60]:
# # load the COCO class labels our YOLO model was trained on
# labelsPath = os.path.sep.join([config.MODEL_PATH, "coco.names"])
# LABELS = open(labelsPath).read().strip().split("\n")
labelsPath = os.path.sep.join([MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

In [61]:
# print(LABELS)

In [62]:
# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([MODEL_PATH, "yolov3.cfg"])

### Using OpenCV’s DNN module, we load our YOLO net into memory. 

In [63]:
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

[INFO] loading YOLO from disk...


If you have the USE_GPU option set in the config, then the backend processor is set to be your NVIDIA CUDA-capable GPU. If you don’t have a CUDA-capable GPU, ensure that the configuration option is set to False so that your CPU is the processor used.

In [64]:
# check if we are going to use GPU
#if USE_GPU == True:
	# set CUDA as the preferable backend and target
#print("[INFO] setting preferable backend and target to CUDA...")
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


In [65]:
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [66]:
# initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None


[INFO] accessing video stream...


In [67]:
!cp '/content/drive/My Drive/pedestrians.mp4' ./  #(./) 현재위치로

![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/05/social_distance_detector_people_detections.jpg)

In [69]:
from google.colab.patches import cv2_imshow
# loop over the frames from the video stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# resize the frame and then detect people (and only people) in it
	frame = imutils.resize(frame, width=700)
	results = detect_people(frame, net, ln,
		personIdx=LABELS.index("person"))

	# initialize the set of indexes that violate the minimum social
	# distance
	violate = set()

	# ensure there are *at least* two people detections (required in
	# order to compute our pairwise distance maps)
	if len(results) >= 2:
		# extract all centroids from the results and compute the
		# Euclidean distances between all pairs of the centroids
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				# check to see if the distance between any two
				# centroid pairs is less than the configured number
				# of pixels
				if D[i, j] < MIN_DISTANCE:
					# update our violation set with the indexes of
					# the centroid pairs
					violate.add(i)
					violate.add(j)

	# loop over the results
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# extract the bounding box and centroid coordinates, then
		# initialize the color of the annotation
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0) # green

		# if the index pair exists within the violation set, then
		# update the color
		if i in violate:
			color = (0, 0, 255) # red

		# draw (1) a bounding box around the person and (2) the
		# centroid coordinates of the person,
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.circle(frame, (cX, cY), 5, color, 1)

	# draw the total number of social distancing violations on the
	# output frame
	text = "Social Distancing Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	# check to see if the output frame should be displayed to our
	# screen
	if args["display"] > 0:
		# show the output frame
		cv2_imshow(frame)
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

	# if an output video file path has been supplied and the video
	# writer has not been initialized, do so now
	if args["output"] != "" and writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		writer.write(frame)

In [74]:
!mv output.avi 'drive/My Drive/tmp/'

### Limitations and future improvements

1. the first step to improving our social distancing detector is to utilize a proper camera calibration.
2. Secondly, you should consider applying a top-down transformation of your viewing angle![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/05/social_distance_detector_topdown.jpg)
3. My third recommendation is to improve the people detection process.

 OpenCV’s YOLO implementation is quite slow not because of the model itself but because of the additional post-processing required by the model.

To further speedup the pipeline, consider utilizing a Single Shot Detector (SSD) running on your GPU — that will improve frame throughput rate considerably.




